In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3

# ---------- BASE DE DATOS ----------
def conectar_db():
    conn = sqlite3.connect("estudiantes.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS estudiantes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL,
            edad INTEGER,
            correo TEXT
        )
    """)
    conn.commit()
    conn.close()

# ---------- FUNCIONES ----------
def agregar_estudiante():
    nombre = entry_nombre.get().strip()
    edad = entry_edad.get().strip()
    correo = entry_correo.get().strip()

    if not nombre or not edad or not correo:
        messagebox.showwarning("Advertencia", "Todos los campos son obligatorios.")
        return

    try:
        edad = int(edad)
    except ValueError:
        messagebox.showerror("Error", "La edad debe ser un número.")
        return

    conn = sqlite3.connect("estudiantes.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO estudiantes (nombre, edad, correo) VALUES (?, ?, ?)",
                   (nombre, edad, correo))
    conn.commit()
    conn.close()

    limpiar_campos()
    mostrar_estudiantes()
    messagebox.showinfo("Éxito", "Estudiante agregado correctamente.")

def mostrar_estudiantes():
    for row in tree.get_children():
        tree.delete(row)
    conn = sqlite3.connect("estudiantes.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM estudiantes")
    for fila in cursor.fetchall():
        tree.insert("", tk.END, values=fila)
    conn.close()

def eliminar_estudiante():
    seleccionado = tree.selection()
    if not seleccionado:
        messagebox.showwarning("Advertencia", "Seleccione un estudiante para eliminar.")
        return
    id_estudiante = tree.item(seleccionado)["values"][0]

    conn = sqlite3.connect("estudiantes.db")
    cursor = conn.cursor()
    cursor.execute("DELETE FROM estudiantes WHERE id=?", (id_estudiante,))
    conn.commit()
    conn.close()

    mostrar_estudiantes()
    messagebox.showinfo("Eliminado", "Estudiante eliminado correctamente.")

def limpiar_campos():
    entry_nombre.delete(0, tk.END)
    entry_edad.delete(0, tk.END)
    entry_correo.delete(0, tk.END)

# ---------- INTERFAZ GRÁFICA ----------
ventana = tk.Tk()
ventana.title("📘 Registro de Estudiantes")
ventana.geometry("750x500")
ventana.configure(bg="#e8f0fe")

# ---------- ESTILOS ----------
style = ttk.Style()
style.theme_use("clam")

# Colores personalizados
style.configure("TFrame", background="#e8f0fe")
style.configure("TLabel", background="#e8f0fe", font=("Segoe UI", 10))
style.configure("TButton", font=("Segoe UI", 10, "bold"), foreground="white", background="#1976d2")
style.map("TButton",
          background=[("active", "#0d47a1")])

style.configure("Treeview", font=("Segoe UI", 9))
style.configure("Treeview.Heading", font=("Segoe UI", 10, "bold"))

# ---------- ENCABEZADO ----------
frame_titulo = tk.Frame(ventana, bg="#1565c0", height=60)
frame_titulo.pack(fill="x")
titulo = tk.Label(frame_titulo, text="Sistema de Registro de Estudiantes",
                  bg="#1565c0", fg="white", font=("Segoe UI", 16, "bold"))
titulo.pack(pady=10)

# ---------- FORMULARIO ----------
frame_form = ttk.LabelFrame(ventana, text="Datos del Estudiante", padding=10)
frame_form.pack(fill="x", padx=20, pady=10)

ttk.Label(frame_form, text="Nombre:").grid(row=0, column=0, padx=5, pady=5, sticky="e")
entry_nombre = ttk.Entry(frame_form, width=35)
entry_nombre.grid(row=0, column=1, padx=5, pady=5)

ttk.Label(frame_form, text="Edad:").grid(row=1, column=0, padx=5, pady=5, sticky="e")
entry_edad = ttk.Entry(frame_form, width=35)
entry_edad.grid(row=1, column=1, padx=5, pady=5)

ttk.Label(frame_form, text="Correo:").grid(row=2, column=0, padx=5, pady=5, sticky="e")
entry_correo = ttk.Entry(frame_form, width=35)
entry_correo.grid(row=2, column=1, padx=5, pady=5)

# ---------- BOTONES ----------
frame_botones = ttk.Frame(ventana)
frame_botones.pack(fill="x", padx=20, pady=10)

btn_agregar = ttk.Button(frame_botones, text="➕ Agregar", command=agregar_estudiante)
btn_agregar.pack(side="left", padx=5)

btn_eliminar = ttk.Button(frame_botones, text="🗑️ Eliminar", command=eliminar_estudiante)
btn_eliminar.pack(side="left", padx=5)

btn_mostrar = ttk.Button(frame_botones, text="🔄 Actualizar", command=mostrar_estudiantes)
btn_mostrar.pack(side="left", padx=5)

btn_limpiar = ttk.Button(frame_botones, text="🧹 Limpiar", command=limpiar_campos)
btn_limpiar.pack(side="left", padx=5)

# ---------- TABLA ----------
frame_tabla = ttk.Frame(ventana)
frame_tabla.pack(fill="both", expand=True, padx=20, pady=10)

tree = ttk.Treeview(frame_tabla, columns=("ID", "Nombre", "Edad", "Correo"), show="headings")
tree.heading("ID", text="ID")
tree.heading("Nombre", text="Nombre")
tree.heading("Edad", text="Edad")
tree.heading("Correo", text="Correo")
tree.column("ID", width=50, anchor="center")
tree.column("Nombre", width=180)
tree.column("Edad", width=80, anchor="center")
tree.column("Correo", width=200)
tree.pack(fill="both", expand=True)

# Scrollbar
scrollbar = ttk.Scrollbar(frame_tabla, orient="vertical", command=tree.yview)
tree.configure(yscroll=scrollbar.set)
scrollbar.pack(side="right", fill="y")

# ---------- INICIO ----------
conectar_db()
mostrar_estudiantes()
ventana.mainloop()
